# AutoRAG - DataCreation Pipeline

# Module

In [3]:
import warnings
warnings.filterwarnings(action='ignore')
import os
import sys
import dotenv
import nest_asyncio
dotenv.load_dotenv()
# nest_asyncio 적용
nest_asyncio.apply()

import pandas as pd

# AutoRAG
from autorag.parser import Parser
from autorag.chunker import Chunker
from autorag.data.qa.schema import Raw, Corpus
from autorag.data.qa.filter.dontknow import dontknow_filter_rule_based
from autorag.data.qa.generation_gt.llama_index_gen_gt import make_basic_gen_gt,make_concise_gen_gt
from autorag.data.qa.query.llama_gen_query import factoid_query_gen
from autorag.data.qa.sample import random_single_hop

# llamaindex
from llama_index.llms.openai import OpenAI

# RawData Parse

In [9]:
# parser 정의
parser = Parser(data_path_glob="/home/livin/rag_pipeline/AutoRAG/data/*.pdf", project_dir="../result/parse")
parser.start_parsing("../settings.yaml")

[10/07/24 10:16:41] INFO     [parser.py:33] >> Parsing Start...                                        ]8;id=991497;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/autorag/parser.py\parser.py]8;;\:]8;id=972600;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/autorag/parser.py#33\33]8;;\

                    INFO     [base.py:19] >> Running parser - langchain_parse module...                  ]8;id=6361;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/autorag/data/parse/base.py\base.py]8;;\:]8;id=902648;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/autorag/data/parse/base.py#19\19]8;;\

[10/07/24 10:16:58] INFO     [parser.py:40] >> Parsing Done!                                           ]8;id=470084;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/autorag/parser.py\parser.py]8;;\:]8;id=346300;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/autorag/parser.py#40\40]8;;\

In [17]:
# Parse 결과 확인
df = pd.read_parquet("../result/parse/hyundai_upstage/0.parquet")
df

# Chunking

In [23]:
chunker = Chunker.from_parquet(parsed_data_path="../result/parse/hyundai_upstage/0.parquet", project_dir="../result/chunk")
chunker.start_chunking("../settings.yaml")

[10/07/24 10:24:53] INFO     [chunker.py:48] >> Chunking Start...                                     ]8;id=55257;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/autorag/chunker.py\chunker.py]8;;\:]8;id=564516;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/autorag/chunker.py#48\48]8;;\

                    INFO     [base.py:22] >> Running chunker - llama_index_chunk module...               ]8;id=182080;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/autorag/data/chunk/base.py\base.py]8;;\:]8;id=112054;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/autorag/data/chunk/base.py#22\22]8;;\

[10/07/24 10:24:56] INFO     [chunker.py:55] >> Chunking Done!                                        ]8;id=540228;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/autorag/chunker.py\chunker.py]8;;\:]8;id=263503;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/autorag/chunker.py#55\55]8;;\

In [25]:
# Chunk 결과 확인
corpus_df = pd.read_parquet("../result/chunk/hyundai_upstage/0.parquet")
corpus_df

,doc_id,contents,path,start_end_idx,metadata
0,af0d9d7f-a1ca-40a9-9c16-67676f0bc620,파일 제목: hyndai.pdf\n 내용: <h1 id='0' style='font...,/home/livin/rag_pipeline/AutoRAG/data/hyndai.pdf,"[0, 1841]","{'last_modified_datetime': '2024-09-06', 'next..."
1,d982d5c6-41fb-4d96-ad9a-76929fe018b5,파일 제목: hyndai.pdf\n 내용: 포 함 하 여 설 명<br>하 고 있 습...,/home/livin/rag_pipeline/AutoRAG/data/hyndai.pdf,"[1458, 3172]","{'last_modified_datetime': '2024-09-06', 'next..."
2,ba009f64-4655-4b0c-b958-684390d73789,파일 제목: hyndai.pdf\n 내용: . 한 국 교 통 안 전 공 단 자 동 ...,/home/livin/rag_pipeline/AutoRAG/data/hyndai.pdf,"[2909, 4374]","{'last_modified_datetime': '2024-09-06', 'next..."
3,f5a378aa-92f4-4be1-a972-4871077582ed,파일 제목: hyndai.pdf\n 내용: 러 스 터</td><td>4</td></...,/home/livin/rag_pipeline/AutoRAG/data/hyndai.pdf,"[4067, 7508]","{'last_modified_datetime': '2024-09-06', 'next..."
4,0a474ba9-d0d5-45d7-a713-dc6e6e367014,파일 제목: hyndai.pdf\n 내용: 이용한 비상 제동이 필요한 경우 .......,/home/livin/rag_pipeline/AutoRAG/data/hyndai.pdf,"[7117, 8718]","{'last_modified_datetime': '2024-09-06', 'next..."
...,...,...,...,...,...
834,1ee41678-458e-4f7b-a148-ac3a925ddd11,파일 제목: hyndai.pdf\n 내용: 차량으로 견인 시 8-20<br>선루프 ...,/home/livin/rag_pipeline/AutoRAG/data/hyndai.pdf,"[1105764, 1106878]","{'last_modified_datetime': '2024-09-06', 'next..."
835,49cf11d4-a9c3-4cd9-b775-24a117bd4702,파일 제목: hyndai.pdf\n 내용: 및 용량 1-14<br>교차로/건널목에서...,/home/livin/rag_pipeline/AutoRAG/data/hyndai.pdf,"[1106656, 1107960]","{'last_modified_datetime': '2024-09-06', 'next..."
836,9cdec7bf-44f4-46c9-b67f-21d987d2087d,파일 제목: hyndai.pdf\n 내용: 고전압 모터 커텍터(엔진룸) 0-18<b...,/home/livin/rag_pipeline/AutoRAG/data/hyndai.pdf,"[1107759, 1108852]","{'last_modified_datetime': '2024-09-06', 'next..."
837,ed053d51-2334-45a7-b738-9c80d2403657,파일 제목: hyndai.pdf\n 내용: 하이빔 보조 작동 5-95<br>드라이브...,/home/livin/rag_pipeline/AutoRAG/data/hyndai.pdf,"[1108613, 1109837]","{'last_modified_datetime': '2024-09-06', 'next..."


# QA Creation

In [27]:
# load RAW data
raw_df = pd.read_parquet("../result/chunk/hyundai_upstage/0.parquet")
raw_instance = Raw(raw_df)

# load Corpus data
corpus_df = pd.read_parquet("../result/chunk/hyundai_upstage/0.parquet")
corpus_instance = Corpus(corpus_df, raw_instance)

In [51]:
# llm 선언
llm = OpenAI(model="gpt-4o-mini")

# QA 데이터 생성 파이프라인
initial_qa = (
    corpus_instance.sample(random_single_hop, random_state=777, n=20) # 몇개의 corpus를 뽑을지 결정
    .map(
        lambda df: df.reset_index(drop=True), # Drop index
    )
    .make_retrieval_gt_contents() # retrieval_gt_contents 생성
    .batch_apply(
        factoid_query_gen,  # query generation
        llm=llm,
        lang="ko",
    )
    .batch_apply(
        make_basic_gen_gt,  # answer generation (basic)
        llm=llm,
        lang="ko",
    )
    .batch_apply(
        make_concise_gen_gt,  # answer generation (concise)
        llm=llm,
        lang="ko",
    )
    .filter(
        dontknow_filter_rule_based,  # unanswerable filter
        lang="ko",
    )
)

[10/07/24 11:04:35] INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=784409;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=520692;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=293778;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=217065;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=164234;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=567723;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=70300;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=895581;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=944147;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=988052;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

...

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=868523;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=981011;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=291439;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=933258;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[10/07/24 11:04:47] INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=686385;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=107527;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[10/07/24 11:04:48] INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=814693;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=336858;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[10/07/24 11:04:51] INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=10361;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=579228;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

In [52]:
# 생성된 qa 저장
# initial_qa.to_parquet('../result/qa/hyundai_upstage_qa_3.parquet', '../result/qa/hyundai_upstage_corpus_3.parquet')

In [48]:
# 생성된 qa 확인
initial_qa.data

,qid,retrieval_gt,retrieval_gt_contents,query,generation_gt
0,a8de5cef-94ec-4791-b4e5-44058a7eaaf5,[[ea418858-0ae1-4e7c-b25c-5841c0c19460]],[[파일 제목: hyndai.pdf\n 내용: 키 관<br>련 앱에서 스마트폰에 저...,디지털 키를 차량 시스템에 등록하기 위해 필요한 첫 번째 단계는 무엇입니까?,"[디지털 키를 차량 시스템에 등록하기 위해 필요한 첫 번째 단계는 ""차량에 등록된 ..."
1,769aad75-814c-42ae-b470-bf640663a0c1,[[7f693838-530d-4690-87d1-1721e0221438]],[[파일 제목: hyndai.pdf\n 내용: id='5' style='font-s...,타이어 관련 유의 사항은 무엇입니까?,"[타이어 관련 유의 사항은 6-48 페이지에 있습니다., 타이어 관련 유의 사항: ..."
2,ff77faa2-b353-4f4f-be61-791acc86bca3,[[93f482c3-2ea8-4c90-8e53-28cebf0ddc78]],[[파일 제목: hyndai.pdf\n 내용: 조절 레버를 놓으면 고정됩니다.<br...,2열 좌석 조절을 위해 어떤 절차를 따라야 합니까?,[2열 좌석 조절을 위해 다음 절차를 따라야 합니다:\n\n1. 좌석 쿠션의 앞쪽 ...


In [55]:
# Chunk 결과 확인
corpus_df = pd.read_parquet("/home/livin/rag_pipeline/AutoRAG/result/qa/hyundai_upstage_qa_3.parquet")
corpus_df

,qid,query,retrieval_gt,generation_gt
0,563d79d7-7481-4a7a-8e04-7d6b98beab1e,와이퍼 모드 표시 여부는 어떻게 설정합니까?,[[65fdc502-fb14-43f4-9881-77c105151d9a]],[와이퍼 모드 표시 여부는 인포테인먼트 시스템의 홈 화면에서 설정 > 차량 > 클러...
1,14370dc9-39e2-4dfd-81b1-e75871ee63ed,차량의 GPS 위치 정보를 저장하는 방법은 무엇입니까?,[[8b5db17d-a725-4be4-81dc-e0d092e3e51f]],[차량의 GPS 위치 정보를 저장하는 방법은 내비게이션 지도에서 표시되는 차량의 G...
2,1d752a52-3424-4edd-a2b4-2f9066e1191f,와이퍼 블레이드를 교체하는 방법은 무엇입니까?,[[64f6eaa3-f724-4e39-b6b6-961f1bfa6fe1]],[와이퍼 블레이드를 교체하는 방법은 다음과 같습니다:\n\n1. 블레이드를 아래로 ...
3,ff657751-19a1-4d36-aebd-3d3ddeeb58de,차량이 미끄러운 길에서 바퀴가 헛돌 때 어떤 조치를 취해야 합니까?,[[8029ee4a-dde6-4e5a-bf21-927b9ac8cac0]],[차량이 미끄러운 길에서 바퀴가 헛돌 때는 다음과 같은 조치를 취해야 합니다:\n\...
4,24aaf21e-4086-48e7-a47e-74d195e5330f,실외 미러는 어떤 종류의 거울인가요?,[[bbf493f4-06d0-4178-ad9c-904edfe0e9a5]],"[실외 미러는 볼록거울입니다., 볼록거울입니다.]"
5,c159faeb-1d20-4001-adad-2b711b82d457,"와셔액의 성분은 무엇이며, 사용 시 주의해야 할 점은 무엇입니까?",[[9104fafd-abe8-4734-8466-ffdf6439f084]],"[와셔액은 알코올 성분을 함유하고 있으며, 유독 성분도 포함되어 있습니다. 사용 시..."
6,b14ed88e-c606-4610-be14-3564d1c60639,디지털 키와 프로필 연동을 통한 개인화 기능은 어떻게 설정하나요?,[[6970336a-0c28-4712-89d3-42b8e2bf7d81]],[디지털 키와 프로필 연동을 통한 개인화 기능은 인포테인먼트 시스템의 설정 > 사용...
7,9905f40d-6bd6-45aa-819e-f8c029721434,전방 레이더가 차량 및 보행자를 인식하지 못할 수 있는 경우는 무엇입니까?,[[7db33de8-4659-4b77-ae9b-6e7e2f4580e0]],[전방 레이더가 차량 및 보행자를 인식하지 못할 수 있는 경우는 다음과 같습니다:\...
8,c9f99638-6472-469c-87c0-777d51653d49,원격 스마트 주차 보조가 정상적으로 작동하지 않을 수 있는 조건은 무엇입니까?,[[cc86a4bc-b229-4372-bc11-1241e8a9c955]],[원격 스마트 주차 보조가 정상적으로 작동하지 않을 수 있는 조건은 다음과 같습니다...
9,584c1757-31c7-4a21-93b5-be38dac1b920,주차 거리 경고가 정상적으로 작동하지 않을 수 있는 경우는 무엇입니까?,[[89849e08-4f55-4855-85e3-9528f4b8585f]],[주차 거리 경고가 정상적으로 작동하지 않을 수 있는 경우는 다음과 같습니다:\n\...


# QA Integration
* 다양한 QA 셋을 통합하여 하나의 QA 셋으로 만듦

In [7]:
qa_list = ["../result/qa/hyundai_upstage_qa_1.parquet", "../result/qa/hyundai_upstage_qa_2.parquet", "../result/qa/hyundai_upstage_qa_3.parquet"]

In [8]:
# 여러 qa 파일을 하나의 qa 파일로 합치기
qa_df = pd.concat([pd.read_parquet(qa) for qa in qa_list], ignore_index=True)

# qa 저장
qa_df.to_parquet("../result/qa/hyundai_upstage_qa_all.parquet")